In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [ ]:
!pip3 install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip3 install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=56b408636a477bad65e415af81099551a86519011b2b270ec7e0be109a4e93fe
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip3 install transformers[torch]

In [ ]:
import accelerate

accelerate.__version__

'0.26.1'

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset

In [ ]:
notebook_login()

In [ ]:
dataset = load_dataset("gopalkalpande/bbc-news-summary", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset["train"][100]

{'File_path': 'sport',
 'Articles': 'Downing injury mars Uefa victory..Middlesbrough boss Steve McClaren hopes that Stewart Downing did not seriously damage his hamstring during the 2-1 Uefa Cup win over Graz AK...The winger was carried off on a stretcher in the closing stages as Boro booked their place in the last 16. McClaren said: "Stewart has tweaked a hamstring, which is a blow for us. We will assess it in the morning. "But I\'m delighted with the match result. They say in sport you can win ugly - and we certainly did." McClaren was relieved to make it through after a tough encounter at The Riverside..."To be fair to Graz, they had a gameplan - they knocked the ball forward, especially in the first half, and we struggled to deal with it. "In the second half we were better but it was frustrating because we got the goal - but one slip and they were back in it." Boro face Sporting Lisbon for a place in the quarter-finals after the Portuguese side saw off Feyenoord on Thursday. And Mc

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Articles"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["Summaries"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1779 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bbc_news_summarization",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.480068,0.201200,0.143600,0.182500,0.182700,19.000000
2,No log,0.362090,0.197700,0.142000,0.181900,0.181700,19.000000
3,No log,0.339155,0.197900,0.141800,0.181500,0.181300,19.000000
4,No log,0.333804,0.198700,0.142700,0.182500,0.182500,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=448, training_loss=0.6783394813537598, metrics={'train_runtime': 472.2517, 'train_samples_per_second': 15.068, 'train_steps_per_second': 0.949, 'total_flos': 1795012407656448.0, 'train_loss': 0.6783394813537598, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1705835750.12e7d7f04521.294.0:   0%|          | 0.00/7.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KishalayGhoshKIIT/bbc_news_summarization/commit/c74a150b8c53bc65d00909b826cc6db25b25bcb9', commit_message='End of training', commit_description='', oid='c74a150b8c53bc65d00909b826cc6db25b25bcb9', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
text = "Thirteen school children aged nine and 10 are confirmed to have died after a fire broke out in a school dormitory in central China, the country's Xinhua state news agency reported. The deadly blaze happened on Friday night at the Yingcai School in Yanshanpu village, Henan province. The manager of the school, near Nanyang city, has been detained and an investigation is under way. One other person is being treated in hospital and is in a stable condition. A teacher at the school told state-run Hebei Daily that all the victims were from the same third-grade class. About 30 students were in the dormitory when the fire broke and the remaining students were safely evacuated. The fire was extinguished less than an hour after firefighters were alerted, Xinhua reported. No further details about the identities of the dead or the cause of the fire were released via official channels. Photos on state media show some windows of the dormitory building have been smashed, and the area around the school cordoned off, with police officers patrolling the scene. According to Xinhua, the school is a privately-run school that had been operating for ten years, and caters for nursery and primary age pupils living in nearby rural areas. The agency reported that children enrolled in the school's nursery had already returned home for the weekend. Fatal fires in China are not uncommon due to lax enforcement of building and safety standards. In November, 26 people died after a large fire ripped through an office building in Luliang City, Shanxi province. A hospital fire in Beijing last April claimed the lives of at least 29 people - mostly patients - and triggered an investigation which saw 12 people detained by police for questioning. Harrowing footage of the fire showed people climbing out of windows onto air conditioning units to escape the flames."

In [2]:
from transformers import pipeline
import json
summarizer = pipeline("summarization", model="KishalayGhoshKIIT/bbc_news_summarization")
js=summarizer(text)
print(js[0])
#js=str(js[0])
#js = js.replace("\'", "\"")
#js=json.loads(js)
print(str(js[0]['summary_text']))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

{'summary_text': "Thirteen school children aged nine and 10 are confirmed to have died after a fire broke out in a school dormitory in central China, the country's Xinhua state news agency reported. The deadly blaze happened on Friday night at the Yingcai School in Yanshanpu village, Henan province. The manager of the school, near Nanyang city, has been detained and an investigation is under way."}
Thirteen school children aged nine and 10 are confirmed to have died after a fire broke out in a school dormitory in central China, the country's Xinhua state news agency reported. The deadly blaze happened on Friday night at the Yingcai School in Yanshanpu village, Henan province. The manager of the school, near Nanyang city, has been detained and an investigation is under way.
